![](https://img.kaikeba.com/web/kkb_index/img_index_logo.png)
# 基础课第一部分（python）第七次作业

各位同学大家好！今天课上演示爬虫的原理，我们来回顾一下爬虫的思路，进行爬虫练习。
爬虫是一个程序，这个程序可以获得网页数据。
## 爬虫的思路
- 1.首先确定需要爬取的网URL地址   
[空气质量指数(http://www.tianqihoubao.com/aqi/)](http://www.tianqihoubao.com/aqi/)    


- 2.通过HTTP/HTTPS协议来获取对应的HTML页面  


- 3.提取HTML页面内有用的数据：
- a. 如果是需要的数据--保存
- b. 如果有其他URL，继续执行第二步

## 爬虫练习 
爬虫项目整体代码：  
[高民权_中国城市空气质量数据抓取_Github](https://github.com/fortyMiles/ChineseAirConditionCrawler)  
【没有头绪的指令】Github中的`get_location_info.py`文件对应city_coding的生成  

**处理城市编码**  
将`<div class="citychk">`copy下来，进一步处理，生成 city_coding  

In [15]:
# 请大家尽量尝试自己从网页中搜寻并copy，而不是直接从课件copy

# 借鉴高老师Github中的get_location_info.py
import requests
from bs4 import BeautifulSoup


def get_location_html():
    http_url = 'http://www.tianqihoubao.com/aqi/'
    r = requests.get(http_url)
    r.encoding = 'GBK'
    html_file = r.text
    return html_file

# print(get_location_html())

def parse_location(html_doc):
    soup = BeautifulSoup(html_doc, 'html.parser')
    CITY_CHECK = '.citychk > dl > dd > a'

    city_map = []

    city_check = soup.select(CITY_CHECK)
#     print(city_check)

    for c in city_check:
#         print(c,'\n')
#         print(c['href'],'\n')
        coding = c['href'].split('/')[2].replace('.html', '')
        city = c.string
        if len(city) > 7 or len(city) == 0 : continue
        city_map.append((city, coding))

    return city_map

# parse_location(get_location_html())

def save_city_map(city_map):
    with open('./city_coding.txt', 'w+') as f:
        for c in city_map:
            if c:
                f.write(c[0] + '\t' + c[1] + '\n')

    print('write done!')

city_coding = parse_location(get_location_html())    
print(len(city_coding), len(set(city_coding)))
    
save_city_map(city_coding)

373 369
write done!


**首先安装包**  

``` bash
pip install bs4
```
参考：[Beautiful Soup 4.2.0 文档](https://www.crummy.com/software/BeautifulSoup/bs4/doc/index.zh.html)

** 读取city_coding **

In [18]:
def get_city_coding(file='./city_coding.txt'):
    #your code
    
    city_coding = {}
    with open(file) as f:
        for line in f.readlines():
            line = line.strip()
            try: 
                city, coding = line.split('\t')
                city_coding[city.strip()] = coding.strip()
            except Exception as e:
                continue
    return city_coding


city_coding = get_city_coding()
# print(city_coding)

{'北京': 'beijing', '天津': 'tianjin', '上海': 'shanghai', '重庆': 'chongqing', '广州': 'guangzhou', '深圳': 'shenzhen', '杭州': 'hangzhou', '成都': 'chengdu', '石家庄': 'shijiazhuang', '唐山': 'tangshan', '秦皇岛': 'qinhuangdao', '保定': 'baoding', '张家口': 'zhangjiakou', '邯郸': 'handan', '邢台': 'xingtai', '承德': 'chengde', '沧州': 'cangzhou', '廊坊': 'langfang', '衡水': 'hengshui', '太原': 'taiyuan', '大同': 'datong', '阳泉': 'yangquan', '长治': 'changzhi', '临汾': 'linfen', '晋城': 'jincheng', '朔州': 'shuozhou', '运城': 'sxyuncheng', '忻州': 'xinzhou', '吕梁': 'lvliang', '晋中': 'jinzhong', '呼和浩特': 'huhehaote', '包头': 'baotou', '鄂尔多斯': 'eerduosi', '乌海': 'wuhai', '赤峰': 'chifeng', '通辽': 'tongliao', '巴彦淖尔': 'bayannaoer', '兴安盟': 'xinganmeng', '阿拉善盟': 'alashanmeng', '呼伦贝尔': 'hulunbeier', '二连浩特': 'erlianhaote', '锡林郭勒': 'xilinguole', '沈阳': 'shenyang', '大连': 'dalian', '丹东': 'dandong', '营口': 'yingkou', '盘锦': 'panjin', '葫芦岛': 'huludao', '鞍山': 'anshan', '锦州': 'jinzhou', '本溪': 'benxi', '瓦房店': 'wafangdian', '抚顺': 'fushun', '辽阳': 'liaoyang', '阜新': 'fuxin

** 指定城市的URL地址确定 **
- 如何拼接成自己想要的URL地址？  
  如果是当前月份可以看到直接使用城市名称即可，如 http://www.tianqihoubao.com/aqi/hangzhou.html  
  如果查询的是历史月份，可以看到是这种格式 http://www.tianqihoubao.com/aqi/hangzhou-201702.html   

In [19]:
def build_url(city_coding, year=None, month=None):#输入城市 年 月 输出 url地址
    BASE = 'http://www.tianqihoubao.com/aqi/'
    city_base_url = BASE + '{}.html'
    city_data_base_url = BASE + '{}-{}{}.html'
    
    if year is not None and month is not None:
        month = str(month) if month >= 10 else '0' + str(month)
        return city_data_base_url.format(city_coding, year, month)
    else:
        return city_base_url.format(city_coding)
    
hangzhou = city_coding['杭州']
print(build_url(hangzhou))
print(build_url(hangzhou, 2018, 5))

http://www.tianqihoubao.com/aqi/hangzhou.html
http://www.tianqihoubao.com/aqi/hangzhou-201805.html


** HTTP 请求状态了解 **
- 200 - 请求成功
- 404 - 请求的资源（网页等）不存在
- 403 - 服务器理解请求客户端的请求，但是拒绝执行此请求     

** 模拟浏览器发送http请求 **
- get post

** 获得响应的数据 分析 保存**

In [30]:
import csv
import os


def parse(url, city_name, city_ENname):
    # your code
    response = requests.get(url)
    if response.ok:
        html = response.text
        soup = BeautifulSoup(html)
        data_table = soup.table

        name_index = 1
        content = data_table.contents[name_index:]
#         print(content)

        result = []
        for index, c in enumerate(content[::2]):
            if index == 0:
                result.append(tuple(['city'] + c.text.split()))
            else:
                result.append(tuple([city_ENname] + c.text.split()))
        
        return result
    else:
        print('Network Error:', response.status_code)

def city_ENname(want_city):
    for i in city_coding:
        if city_coding[i] == want_city :
            return i
        
city_coding = get_city_coding()
want_city = city_coding['杭州']
city_ENname = city_ENname(want_city)
url = build_url(want_city, 2019, 9)
result = parse(url, want_city, city_ENname)
print(result)

def save_csv(file, data):
    # your code 
    if data == None or len(data) == 1: return
    if os.path.exists(file):
        with open(file, 'a') as f:
            writer = csv.writer(f)
            writer.writerows(data[1:])
    else:
        with open(file, 'w') as f:
            writer = csv.writer(f)
            writer.writerows(data)

save_csv(f'./{want_city}.csv', result)

[('city', '日期', '质量等级', 'AQI指数', '当天AQI排名', 'PM2.5', 'PM10', 'So2', 'No2', 'Co', 'O3'), ('杭州', '2019-09-01', '优', '27', '66', '12', '18', '4', '24', '0.64', '80'), ('杭州', '2019-09-02', '优', '22', '52', '9', '15', '4', '35', '0.72', '32'), ('杭州', '2019-09-03', '优', '30', '104', '17', '29', '4', '41', '0.73', '15'), ('杭州', '2019-09-04', '优', '49', '238', '32', '50', '5', '48', '0.98', '27'), ('杭州', '2019-09-05', '优', '34', '124', '19', '33', '4', '33', '0.77', '34'), ('杭州', '2019-09-06', '优', '23', '40', '11', '22', '4', '23', '0.60', '36'), ('杭州', '2019-09-07', '优', '35', '84', '18', '32', '4', '24', '0.62', '74'), ('杭州', '2019-09-08', '良', '80', '288', '46', '78', '5', '43', '0.85', '112'), ('杭州', '2019-09-09', '良', '78', '309', '47', '77', '5', '44', '0.88', '100'), ('杭州', '2019-09-10', '良', '79', '344', '50', '84', '6', '49', '0.83', '103'), ('杭州', '2019-09-11', '良', '71', '344', '41', '70', '5', '39', '0.75', '100'), ('杭州', '2019-09-12', '良', '57', '299', '32', '52', '5', '26', '0.6

** 整体流程 **

In [ ]:
def crawler_all():
    file = './data/allcity_2019.csv'
    
    city_codings = get_city_coding()
    allcities = list(city_codings.keys())
    
    for city in allcities:
        city_code = city_codings[city]
        for year in range(2019,2012,-1):
            for month in range(1,13):
                url = build_url(city_code, year, month)
                result = parse(url, city_code) # city
                print(f'\r{city} {year}-{month} {len(result)}', end='')
                save_csv(file, result)
                time.sleep(1)

if __name__ == '__main__':
    crawler_all()